In [1]:
import pandas as pd
import requests
import json
import urllib.request
import time
from bs4 import BeautifulSoup
import re
import numpy as np

In [2]:
from rank_bm25 import BM25Okapi
from nltk.tokenize import word_tokenize

In [3]:
data = pd.read_csv("../Eclipse/final_eclipse_data_15thmarch.csv")

In [4]:
sentences = data['Description']

In [5]:
data

,Bug_ID,Product,Component,Status,Resolution,Summary,Changed,Description,Duplicate_Bug_Ids
0,518088.0,Web Tools,Web Standard Tools,RESOLVED,INVALID,search widget is not working,2017-06-10 14:42:10,search widget is not working,NaN
1,546444.0,z_Archived,PDT,CLOSED,NOT_ECLIPSE,Bug,2020-05-14 11:25:40,created attachment eclipse hello world,NaN
2,533893.0,Platform,Website,RESOLVED,INVALID,AntCompareCVSDebugDocIDEIncubatorPMCRelengReso...,2018-04-22 12:23:23,created attachment error in website antcomp...,NaN
3,519449.0,PDE,UI,VERIFIED,FIXED,Problem with KEY_NAME,2017-08-03 03:21:28,in product pluginsection recommendedbundles m...,NaN
4,519450.0,JDT,UI,RESOLVED,FIXED,Problem with KEY_NAME,2017-07-13 09:43:02,in renametyperefactoring another type the tem...,NaN
...,...,...,...,...,...,...,...,...,...
49239,543844.0,Equinox,Framework,RESOLVED,FIXED,FrameworkExtensionInstaller in dev mode is una...,2019-01-25T16:49:13Z,if string devpaths configuration get...,NaN
49240,541635.0,Platform,SWT,VERIFIED,FIXED,[Wayland] Random behaviour when moving editor ...,2018-11-28T12:52:44Z,hello first thank you for your last fixes a...,NaN
49241,555559.0,Capella,UI,RESOLVED,FIXED,Double-clicking on an Element should not fold/...,2017-08-25T13:34:38Z,double clicking on an element should not fold ...,NaN
49242,518157.0,JDT,Core,VERIFIED,FIXED,A Class<? extends RawType> = Impl.class where ...,2017-06-13T00:58:14Z,overview this is a new bug in oxygen neo...,NaN


In [6]:
tokens = []
#sentences="helloo dear, sdlm skdj "
for s in sentences:
    cleanedTex=re.sub(r'[^\w\s]','',str(s)).lower()
    words = (word_tokenize(cleanedTex))
    tokens .append(words)

In [7]:
bm25 = BM25Okapi(tokens)

In [8]:
data = data.rename(columns = {"Bug_ID":"Bug_Id"})

In [9]:
nan_value = float("NaN")
data.replace("", nan_value, inplace=True)
data.dropna(subset = ["Description"], inplace=True)
cond = (data['Description'] == 'NaN')
cond.unique()
data = data.reset_index(drop=True)

In [10]:
data['Description']

0                             search widget is not working
1              created attachment   eclipse   hello world 
2        created attachment   error in website  antcomp...
3        in  product pluginsection recommendedbundles m...
4        in  renametyperefactoring another type the tem...
                               ...                        
49239    if       string   devpaths   configuration get...
49240    hello   first  thank you for your last fixes a...
49241    double clicking on an element should not fold ...
49242        overview  this is a new bug in oxygen  neo...
49243    hi   we are facing an issue in jakartaee tck p...
Name: Description, Length: 49244, dtype: object

In [13]:
data['Description']=data['Description'].values.astype('object')

### On complete data

In [14]:
count=0
dup_indices_with_bug_id = {}
similarity_scores_before = []
similarity_scores = {}
issues = data.Bug_Id

for i, row in data.iterrows():
    
    issue_id = row.Bug_Id
    dup_id = row.Duplicate_Bug_Ids
    
    if dup_id not in issues.values:
        count+=1
#         print(f'{i}. Duplicate ID {dup_id} not found in issue IDs')
        continue

    similarity_row = bm25.get_scores(tokens[i])
    argsort_similarity = np.argsort(similarity_row)[::-1][1:]
    similarity_scores_before = (np.sort(similarity_row)[::-1][1:])
#     similarity_scores[issue_id] = np.min(similarity_scores_before[np.nonzero(similarity_scores_before)])
    dup_bug_index = list(issues[issues == dup_id].index)[0]
    dup_ranking = np.argmax(argsort_similarity == np.int64(dup_bug_index))
    
    dup_indices_with_bug_id[issue_id] = dup_ranking
# print(count)

In [15]:
count_at_1=0
count_at_5=0
count_at_10=0
count_at_20=0
count_at_25=0
count_at_30=0
count_at_50=0
count_at_75=0
count_at_100=0

for key, value in enumerate(dup_indices_with_bug_id.values()):
    if(value < 1):
        count_at_1+=1
    if(value < 5):
        count_at_5+=1
    if(value < 10):
        count_at_10+=1
    if(value < 20):
        count_at_20+=1
    if(value < 25):
        count_at_25+=1
    if(value < 30):
        count_at_30+=1
    if(value < 50):
        count_at_50+=1
    if(value < 75):
        count_at_75+=1
    if(value < 100):
        count_at_100+=1
        
print("count at 1",count_at_1)        
print("count at 5",count_at_5)
print("count at 10",count_at_10)
print("count at 20",count_at_20)
print("count at 25",count_at_25)
print("count at 30",count_at_30)
print("count at 50",count_at_50)
print("count at 75",count_at_75)
print("count at 100",count_at_100)

N = len(dup_indices_with_bug_id)

recall_rate_at_1 = count_at_1/N
print("recall_rate_at_1",recall_rate_at_1)

recall_rate_at_5 = count_at_5/N
print("recall_rate_at_5",recall_rate_at_5)

recall_rate_at_10 = count_at_10/N
print("recall_rate_at_10",recall_rate_at_10)

recall_rate_at_20 = count_at_20/N
print("recall_rate_at_20",recall_rate_at_20)

recall_rate_at_25 = count_at_25/N
print("recall_rate_at_25",recall_rate_at_25)

recall_rate_at_30 = count_at_30/N
print("recall_rate_at_30",recall_rate_at_30)

recall_rate_at_50 = count_at_50/N
print("recall_rate_at_50",recall_rate_at_50)

recall_rate_at_75 = count_at_75/N
print("recall_rate_at_75",recall_rate_at_75)

recall_rate_at_100 = count_at_100/N
print("recall_rate_at_10",recall_rate_at_100)

count at 1 324
count at 5 924
count at 10 1102
count at 20 1259
count at 25 1313
count at 30 1351
count at 50 1449
count at 75 1543
count at 100 1608
recall_rate_at_1 0.10106051154086089
recall_rate_at_5 0.28820960698689957
recall_rate_at_10 0.3437305053025577
recall_rate_at_20 0.3927011852776045
recall_rate_at_25 0.409544603867748
recall_rate_at_30 0.42139737991266374
recall_rate_at_50 0.4519650655021834
recall_rate_at_75 0.48128509045539614
recall_rate_at_10 0.5015595757953837


## Textually Similar

In [16]:
textually_similar_data = pd.read_csv("../Eclipse/eclipse_textually_similar_duplicates.csv")

In [17]:
textually_similar_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1621 entries, 0 to 1620
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Bug_ID             1621 non-null   float64
 1   Product            1621 non-null   object 
 2   Component          1621 non-null   object 
 3   Status             1621 non-null   object 
 4   Resolution         1621 non-null   object 
 5   Summary            1621 non-null   object 
 6   Changed            1621 non-null   object 
 7   Description        1621 non-null   object 
 8   Duplicate_Bug_Ids  1621 non-null   float64
dtypes: float64(2), object(7)
memory usage: 114.1+ KB


In [18]:
sentence_sim = textually_similar_data.Description

tokens_sim = []
for s in sentence_sim:
    cleanedTex=re.sub(r'[^\w\s]','',str(s)).lower()
    words = (word_tokenize(cleanedTex))
    tokens_sim.append(words)

In [19]:
count=0
dup_indices_with_bug_id_similar = {}
similarity_scores_before = []
similarity_scores = {}
issues = data.Bug_Id

for i, row in textually_similar_data.iterrows():
    
    issue_id = row.Bug_ID
    dup_id = row.Duplicate_Bug_Ids
    
    if dup_id not in issues.values:
        count+=1
#         print(f'{i}. Duplicate ID {dup_id} not found in issue IDs')
        continue
#     if i==4427:
#         break
    similarity_row = bm25.get_scores(tokens_sim[i])
    argsort_similarity = np.argsort(similarity_row)[::-1][1:]
    similarity_scores_before = (np.sort(similarity_row)[::-1][1:])
#     similarity_scores[issue_id] = np.min(similarity_scores_before[np.nonzero(similarity_scores_before)])
    dup_bug_index = list(issues[issues == dup_id].index)[0]
    dup_ranking = np.argmax(argsort_similarity == np.int64(dup_bug_index))
    
    dup_indices_with_bug_id_similar[issue_id] = dup_ranking
# print(count)

In [20]:
count_at_1=0
count_at_5=0
count_at_10=0
count_at_20=0
count_at_25=0
count_at_30=0
count_at_50=0
count_at_75=0
count_at_100=0

for key, value in enumerate(dup_indices_with_bug_id_similar.values()):
    if(value < 1):
        count_at_1+=1
    if(value < 5):
        count_at_5+=1
    if(value < 10):
        count_at_10+=1
    if(value < 20):
        count_at_20+=1
    if(value < 25):
        count_at_25+=1
    if(value < 30):
        count_at_30+=1
    if(value < 50):
        count_at_50+=1
    if(value < 75):
        count_at_75+=1
    if(value < 100):
        count_at_100+=1
        
print("count at 1",count_at_1)        
print("count at 5",count_at_5)
print("count at 10",count_at_10)
print("count at 20",count_at_20)
print("count at 25",count_at_25)
print("count at 30",count_at_30)
print("count at 50",count_at_50)
print("count at 75",count_at_75)
print("count at 100",count_at_100)

N = len(dup_indices_with_bug_id_similar)

recall_rate_at_1 = count_at_1/N
print("recall_rate_at_1",recall_rate_at_1)

recall_rate_at_5 = count_at_5/N
print("recall_rate_at_5",recall_rate_at_5)

recall_rate_at_10 = count_at_10/N
print("recall_rate_at_10",recall_rate_at_10)

recall_rate_at_20 = count_at_20/N
print("recall_rate_at_20",recall_rate_at_20)

recall_rate_at_25 = count_at_25/N
print("recall_rate_at_25",recall_rate_at_25)

recall_rate_at_30 = count_at_30/N
print("recall_rate_at_30",recall_rate_at_30)

recall_rate_at_50 = count_at_50/N
print("recall_rate_at_50",recall_rate_at_50)

recall_rate_at_75 = count_at_75/N
print("recall_rate_at_75",recall_rate_at_75)

recall_rate_at_100 = count_at_100/N
print("recall_rate_at_10",recall_rate_at_100)

count at 1 160
count at 5 442
count at 10 530
count at 20 625
count at 25 662
count at 30 687
count at 50 743
count at 75 791
count at 100 824
recall_rate_at_1 0.1005656819610308
recall_rate_at_5 0.27781269641734757
recall_rate_at_10 0.3331238214959145
recall_rate_at_20 0.3928346951602766
recall_rate_at_25 0.41609050911376494
recall_rate_at_30 0.431803896920176
recall_rate_at_50 0.46700188560653677
recall_rate_at_75 0.497171590194846
recall_rate_at_10 0.5179132620993087


# Textually Dissimilar

In [32]:
textually_dissimilar_data = pd.read_csv("../Eclipse/eclipse_textually_dissimilar_duplicates.csv")

In [38]:
textually_dissimilar_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1622 entries, 0 to 1621
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Bug_ID             1622 non-null   float64
 1   Product            1622 non-null   object 
 2   Component          1622 non-null   object 
 3   Status             1622 non-null   object 
 4   Resolution         1622 non-null   object 
 5   Summary            1622 non-null   object 
 6   Changed            1622 non-null   object 
 7   Description        1622 non-null   object 
 8   Duplicate_Bug_Ids  1622 non-null   float64
dtypes: float64(2), object(7)
memory usage: 114.2+ KB


In [39]:
sentence_dissim = textually_dissimilar_data.Description

tokens_dissim = []
for s in sentence_dissim:
    cleanedTex=re.sub(r'[^\w\s]','',str(s)).lower()
    words = (word_tokenize(cleanedTex))
    tokens_dissim.append(words)

In [40]:
count=0
dup_indices_with_bug_id_dissimilar = {}
similarity_scores_before = []
similarity_scores = {}

issues = data.Bug_Id

for i, row in textually_dissimilar_data.iterrows():
    
    issue_id = row.Bug_ID
    dup_id = row.Duplicate_Bug_Ids
    
    if dup_id not in issues.values:
        count+=1
#         print(f'{i}. Duplicate ID {dup_id} not found in issue IDs')
        continue
    if i== 2292:
        break
    similarity_row = bm25.get_scores(tokens_dissim[i])
    argsort_similarity = np.argsort(similarity_row)[::-1][1:]
    similarity_scores_before = (np.sort(similarity_row)[::-1][1:])
#     similarity_scores[issue_id] = np.min(similarity_scores_before[np.nonzero(similarity_scores_before)])
    dup_bug_index = list(issues[issues == dup_id].index)[0]
    dup_ranking = np.argmax(argsort_similarity == np.int64(dup_bug_index))
    
    dup_indices_with_bug_id_dissimilar[issue_id] = dup_ranking
# print(count)

In [41]:
count_at_1=0
count_at_5=0
count_at_10=0
count_at_20=0
count_at_25=0
count_at_30=0
count_at_50=0
count_at_75=0
count_at_100=0

for key, value in enumerate(dup_indices_with_bug_id_dissimilar.values()):
    if(value < 1):
        count_at_1+=1
    if(value < 5):
        count_at_5+=1
    if(value < 10):
        count_at_10+=1
    if(value < 20):
        count_at_20+=1
    if(value < 25):
        count_at_25+=1
    if(value < 30):
        count_at_30+=1
    if(value < 50):
        count_at_50+=1
    if(value < 75):
        count_at_75+=1
    if(value < 100):
        count_at_100+=1
        
print("count at 1",count_at_1)        
print("count at 5",count_at_5)
print("count at 10",count_at_10)
print("count at 20",count_at_20)
print("count at 25",count_at_25)
print("count at 30",count_at_30)
print("count at 50",count_at_50)
print("count at 75",count_at_75)
print("count at 100",count_at_100)

N = len(dup_indices_with_bug_id_dissimilar)

recall_rate_at_1 = count_at_1/N
print("recall_rate_at_1",recall_rate_at_1)

recall_rate_at_5 = count_at_5/N
print("recall_rate_at_5",recall_rate_at_5)

recall_rate_at_10 = count_at_10/N
print("recall_rate_at_10",recall_rate_at_10)

recall_rate_at_20 = count_at_20/N
print("recall_rate_at_20",recall_rate_at_20)

recall_rate_at_25 = count_at_25/N
print("recall_rate_at_25",recall_rate_at_25)

recall_rate_at_30 = count_at_30/N
print("recall_rate_at_30",recall_rate_at_30)

recall_rate_at_50 = count_at_50/N
print("recall_rate_at_50",recall_rate_at_50)

recall_rate_at_75 = count_at_75/N
print("recall_rate_at_75",recall_rate_at_75)

recall_rate_at_100 = count_at_100/N
print("recall_rate_at_10",recall_rate_at_100)

count at 1 164
count at 5 482
count at 10 572
count at 20 634
count at 25 651
count at 30 664
count at 50 706
count at 75 752
count at 100 784
recall_rate_at_1 0.10154798761609907
recall_rate_at_5 0.29845201238390096
recall_rate_at_10 0.3541795665634675
recall_rate_at_20 0.3925696594427245
recall_rate_at_25 0.40309597523219814
recall_rate_at_30 0.41114551083591333
recall_rate_at_50 0.4371517027863777
recall_rate_at_75 0.46563467492260063
recall_rate_at_10 0.4854489164086687
